# Contoso Sales Analysis Assistant

The following notebook includes a version of [demo 3](./demo-3-contoso-sales-analysis.ipynb) which uses [Azure AI Agent Service](https://techcommunity.microsoft.com/blog/azure-ai-services-blog/introducing-azure-ai-agent-service/4298357) to build the Contoso Sales Assistant.

## Azure AI Agent Service
Azure AI Agent service builds upon Azure OpenAI Assistants API to provide an **enterprise-grade** solution for building scalable agents safely and securely. It integrates with an **extensive ecosystem of tools** to enable agents to ground their knowledge in real-time data (e.g. Azure AI Search, Sharepoint and Bing Search) and to **act on behalf of users** (e.g. Logic Apps and Azure Functions).
Azure AI Agent Service also allows a flexible model choice, going beyond the OpenAI collection. 

The service is accessible through the [Azure AI Foundry SDK](https://techcommunity.microsoft.com/blog/aiplatformblog/ignite-2024-announcing-the-azure-ai-foundry-sdk/4295862), which provides a simplified coding experience to build AI applications. 

> [!IMPORTANT]
> The Azure AI Agent Service is currently in **private preview**. This early stage of development means the product is actively evovling, with significant updates and improvements expected. Users should anticipate changes as we work towards refining features, enhancing functionality, and expanding capabilities. We welcome feedback and contributions during this phase to help shape the future of the product.
[Join the waitlist](https://nam.dcv.ms/nzy5CEG6Br) to get access to the private preview.

## Installation

Refer to the README.md file in this folder for installation instructions.

### Load parameters

In [ ]:
import os
import json
import sqlite3
from datetime import datetime
from typing import Any, Callable, Iterable, Set

from PIL import Image
from IPython.display import display
import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    CodeInterpreterTool,
    FunctionTool,
    MessageImageFileContent,
    MessageTextContent,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ThreadMessage,
    ThreadRun,
    ToolOutput,
    ToolSet,
)

from sales_data import SalesData

load_dotenv(".env")

API_DEPLOYMEMT_NAME = os.getenv("OPENAI_GPT_DEPLOYMENT")

thread = None
agent = None

### Import Libraries and open the contoso-sales SQLite database

In [ ]:
# con = sqlite3.connect("./database/contoso-sales.db")

sales_data = SalesData()
sales_data.connect()
db_info = sales_data.get_database_info()

### Create an AI Project client

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

### Define the Agent tools

The are three tools defined:

1. code_interpreter
1. `fetch_sales_data_using_sqlite_query`: This function returns the sales from the SQLite database.


In [ ]:
user_functions: Set[Callable[..., Any]] = {
    sales_data.fetch_sales_data_using_sqlite_query,
}

In [ ]:
functions = FunctionTool(functions=user_functions)
code_interpreter = CodeInterpreterTool()

toolset = ToolSet()
toolset.add(functions)
toolset.add(code_interpreter)

### Set the Assistant instruction context

Sets the context for the conversation. The instructions are equivalent to setting the system message for an OpenAI chat completion.

In [ ]:
instructions = (
    "You are an advanced sales analysis assistant for Contoso. Your role is to assist users with their sales data inquiries while maintaining a polite, professional, helpful, and friendly demeanor.",
    "Use the `fetch_sales_data_using_sqlite_query` function for sales data queries, defaulting to aggregated data unless a detailed breakdown is requested. The function returns JSON data.",
    f"Reference the following SQLite schema for the Contoso sales database: {db_info}.",
    "If asked for 'help,' suggest example queries (e.g., 'What was last quarter's revenue?' or 'Top-selling products in Europe?').",
    "Only use data from the Contoso sales database to respond. If the query falls outside the available data or your expertise, or you're unsure, reply with: I'm unable to assist with that. Please ask more specific questions about Contoso sales and products or contact IT for further help.",

    "Responsibilities:",
    "1. Data Analysis: Provide insights based on available sales data.",
    "2. Visualizations: Generate charts or graphs to illustrate trends.",
    "3. Scope Awareness:",
    "   - For non-sales-related questions, respond:",
    "     'I'm unable to assist with that. Please contact IT for more assistance.'",
    "   - For help requests, provide example questions you can answer.",
    "4. Handling Difficult Interactions:",
    "   - Remain calm and professional with upset or insulting users.",
    "   - Respond: 'I'm here to help with your sales data inquiries. If you need further assistance, please contact IT.'",

    "Tone & Conduct:",
    "- Maintain a professional and courteous tone.",
    "- Avoid sharing sensitive or confidential information.",

    f"The current date and time is: {datetime.now().strftime('%x %X')}."
)

### Process Function calling

Loops through the conversation and calls the appropriate function based on the user input.

In [ ]:
def call_functions(project_client: AIProjectClient, thread_id: str, run: ThreadRun) -> None:
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    if not tool_calls:
        print("No tool calls provided - cancelling run")
        project_client.agents.cancel_run(thread_id=thread_id, run_id=run.id)
        return

    tool_outputs = []
    for tool_call in tool_calls:
        if isinstance(tool_call, RequiredFunctionToolCall):
            try:
                print(f"Executing tool call: {tool_call}")
                output = functions.execute(tool_call)
                tool_outputs.append(
                    ToolOutput(
                        tool_call_id=tool_call.id,
                        output=output,
                    )
                )
            except Exception as e:
                print(f"Error executing tool_call {tool_call.id}: {e}")

    print(f"Tool outputs: {tool_outputs}")
    if tool_outputs:
        project_client.agents.submit_tool_outputs_to_run(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
        )
    print(f"Run finished with status: {run.status}")

### Format and display the Agent Messages for text and images

Utility functions to format and display the Agent messages.

In [ ]:
def format_messages(messages: Iterable[ThreadMessage]) -> None:
    last_msg = messages.get_last_message_by_sender("assistant")

    if last_msg:
        for content in last_msg.content:
            if isinstance(content, MessageTextContent):
                print(f"Last Message: {content.text.value}")

            if isinstance(content, MessageImageFileContent):
                    print(f"Image File ID: {content.image_file.file_id}")
                    file_name = f"{content.image_file.file_id}_image_file.png"
                    project_client.agents.save_file(file_id=content.image_file.file_id, file_name=file_name)
                    image = Image.open(file_name)
                    image = image.resize((image.width // 2, image.height // 2), Image.LANCZOS)
                    project_client.agents.delete_file(content.image_file.file_id)
                    display(image)

### Process the user messages

Loops through the conversation and calls the appropriate function based on the user input.

In [ ]:
def process_message(content: str) -> None:
    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=content,
    )
    print(f"Created message, ID: {message.id}")

    run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
           call_functions(project_client, thread.id, run) 

    # Fetch and log all messages
    messages = project_client.agents.get_messages(thread_id=thread.id)
    format_messages(messages)


### Create an Agent Object

The Agent is responsible for managing the conversation with the user.

In [ ]:
agent = project_client.agents.create_agent(
    model=API_DEPLOYMEMT_NAME, name="my-assistant", instructions="\n".join(instructions), toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

### Create a thread

Threads in the Agent Service are designed to be session-based.
Each thread is a conversation between the user and the assistant.

In [ ]:
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID {thread.id}")

### Have a conversation with the Agent

In [ ]:
process_message("what was the total sales data for region? Display as a table.")

In [ ]:
process_message("Pie chart of sales by region. Use different colors for each region.")

In [ ]:
process_message("Calculate the worldwide sales revenue. Display as a table.")

In [ ]:
process_message("what were the sales for March in 2023 by region? Display as a table.")

## Cleaning up

In [ ]:
project_client.agents.delete_agent(agent.id)
# con.close()
sales_data.close()
print("Deleted agent")